In [97]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [98]:
# Define paths
train_cars_dataset_path = 'data/cars_train/train'
test_cars_dataset_path = 'data/cars_train/test'

classes = os.listdir(train_cars_dataset_path)

# Initialize lists to store images and labels
train_images = []
train_labels = []
test_images = []
test_labels = []

# Loop through each subfolder in the data folder
for label_folder in os.listdir(train_cars_dataset_path):
    train_label_path = os.path.join(train_cars_dataset_path, label_folder)
    test_label_path = os.path.join(test_cars_dataset_path, label_folder)
    if os.path.isdir(train_label_path) and os.path.isdir(test_label_path):
        # Extract label from folder name
        label = classes.index(label_folder)  # Assuming folder names are the labels
        
        # Loop through images in the current label folder
        for image_file in os.listdir(train_label_path):
            train_image_path = os.path.join(train_label_path, image_file)
            if image_file.endswith(".jpg"):  # Assuming images are in JPG format
                # Read image and resize to a fixed size if necessary
                image = cv2.imread(train_image_path)
                image = cv2.resize(image, (224, 224))
                train_images.append(image)
                train_labels.append(label)

        for image_file in os.listdir(test_label_path):
            test_image_path = os.path.join(test_label_path, image_file)
            if image_file.endswith(".jpg"):  # Assuming images are in JPG format
                # Read image and resize to a fixed size if necessary
                image = cv2.imread(test_image_path)
                image = cv2.resize(image, (224, 224))
                test_images.append(image)
                test_labels.append(label)

In [99]:
# Convert lists to numpy arrays
X_train = np.array(train_images)
Y_train = np.array(train_labels)

X_test = np.array(test_images)
Y_test = np.array(test_labels)

# Flatten the image data
X_train = X_train.reshape(X_train.shape[0], -1).T
X_test = X_test.reshape(X_test.shape[0], -1).T

# Normalize pixel values to range [0, 1]
X_train = X_train / 255.0
X_test = X_test / 255.0

num_classes = len(classes)
pixels = len(X_train[:, 0])

In [100]:
def init_params():
    np.random.seed(1)
    W1 = np.random.randn(num_classes, pixels) * 0.01
    b1 = np.zeros((num_classes, 1))
    W2 = np.random.randn(num_classes, num_classes) * 0.01
    b2 = np.zeros((num_classes, 1))
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    exp_Z = np.exp(Z - np.max(Z)) # Numerical stability
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = np.dot(W1, X) + b1
    A1 = ReLU(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * np.dot(dZ2, A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T, dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * np.dot(dZ1, X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 -= alpha * dW1
    b1 -= alpha * db1
    W2 -= alpha * dW2
    b2 -= alpha * db2
    return W1, b1, W2, b2

In [101]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.mean(predictions == Y) * 100

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            predictions = get_predictions(A2)
            accuracy = get_accuracy(predictions, Y)
            print(f"Iteration: {i}, Accuracy: {accuracy:.2f}%")
    return W1, b1, W2, b2

In [102]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, alpha=0.01, iterations=100)

Iteration: 0, Accuracy: 0.62%
Iteration: 10, Accuracy: 0.57%
Iteration: 20, Accuracy: 0.84%
Iteration: 30, Accuracy: 0.84%
Iteration: 40, Accuracy: 0.90%
Iteration: 50, Accuracy: 0.92%
Iteration: 60, Accuracy: 0.93%
Iteration: 70, Accuracy: 0.90%
Iteration: 80, Accuracy: 0.90%
Iteration: 90, Accuracy: 0.95%


In [174]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((224, 224, 3))
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [178]:
dev_predictions = make_predictions(X_test, W1, b1, W2, b2)
print(dev_predictions)
get_accuracy(dev_predictions, Y_test)

[120  22 120 ... 120 120 120]


0.9400705052878966